# To Do

In [ ]:
# 1. Change collect_data() to implement the following dataset structure,
# Dataset -> Compost -> one.jpg, two.jpg, ...
#            Landfill -> one.jpg, two.jpg, ...

# 2. Check if cnn_model() -> layer2 = create_new_conv_layer(..., NUM_FILTERS*2, ...) is correct
# 3. Change constants in cnn_model() -> fully connected layer
# 4. Change IMAGE_SIZE_RESHAPE

In [ ]:
import glob
import cv2
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
from PIL import Image
from random import shuffle

# Parameters

In [ ]:
DATASET_PATH = "Dataset/**/*.jpg"
CLASS_LABELS = ['compost', 'landfill', 'recyclable']
TRAIN_VAL_TEST = {'train': 0.6, 'val': 0.8, 'test': 1} # Train: 60%, Val: 20%, Test: 20%
IMAGE_RESIZE_SHAPE = (28, 28)
IMAGE_RESIZE_SHAPE_LIST = [28, 28, 3]
IMAGE_SIZE = 28
NUM_CHANNELS = 3 # RGB
NUM_CLASSES = 3 # Compost, Landfill & Recyclable
NUM_EPOCHS = 5
BATCH_SIZE = 5
NUM_FILTERS = 32
FILTER_SHAPE = [5, 5]
POOL_SHAPE = [2, 2]

# Collect & Split Data

In [ ]:
def collect_split_data():
    
    labels = []
    
    files = glob.glob(DATASET_PATH)
    for file in files:
        if CLASS_LABELS[0] in file:
            labels.append(0)
        elif CLASS_LABELS[1] in file:
            labels.append(1)
        elif CLASS_LABELS[2] in file:
            labels.append(2)
        else:
            print("Error: Image filename does not contain correct label.")
                 
    c = list(zip(files, labels))
    shuffle(c)
    files, labels = zip(*c)
    
    train_img = files[0:int(TRAIN_VAL_TEST['train'] * len(files))]
    train_labels = labels[0:int(TRAIN_VAL_TEST['train'] * len(files))]
    val_img = files[int(TRAIN_VAL_TEST['train'] * len(files)) : int(TRAIN_VAL_TEST['val'] * len(files))]
    val_labels = labels[int(TRAIN_VAL_TEST['train'] * len(files)) : int(TRAIN_VAL_TEST['val'] * len(files))]
    test_img = files[int(TRAIN_VAL_TEST['val'] * len(files)):]
    test_labels = labels[int(TRAIN_VAL_TEST['val'] * len(files)):]
        
    return len(files), train_img, train_labels, val_img, val_labels, test_img, test_labels

# Write data to tfrecords file

In [ ]:
def load_image(addr):
    # Read, resize and convert to RGB (since cv2 loads images as BGR)
    img = Image.open(addr)
    img = cv2.imread(addr)
    img = cv2.resize(img, IMAGE_RESIZE_SHAPE, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    return img

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def create_tfrecord(files, labels, train_val_test):
        
    # Open .tfrecords file
    writer = tf.python_io.TFRecordWriter(train_val_test+'.tfrecords')
    
    for i in range(len(files)):
    
        # Load image and its label
        img = load_image(files[i])
        label = labels[i]

        # Create a feature
        feature = { train_val_test+'/label': _int64_feature(label),
                    train_val_test+'/image': _bytes_feature(tf.compat.as_bytes(img.tobytes()))}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))

        # Serialize to string and write to file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

def create_tfrecords(train_img, train_labels, val_img, val_labels, test_img, test_labels):
    
    create_tfrecord(train_img, train_labels, 'train')
    create_tfrecord(val_img, val_labels, 'val')
    create_tfrecord(test_img, test_labels, 'test')

# Read data from tfrecords file

In [ ]:
def read_from_tfrecords(train_val_test):
    
    file = glob.glob(train_val_test+'.tfrecords')
    

    feature = { train_val_test+'/image': tf.FixedLenFeature([], tf.string),
                train_val_test+'/label': tf.FixedLenFeature([], tf.int64) }

    # Enqueue train.tfrecords
    filename_queue = tf.train.string_input_producer(file, num_epochs=1)

    # Define reader and read file from queue
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    
    # Convert serialized data back to arrays and numbers
    image = tf.decode_raw(features[train_val_test+'/image'], tf.float32)
    label = tf.cast(features[train_val_test+'/label'], tf.int32)
    
    # Reshape image data to original shape
    image = tf.reshape(image, IMAGE_RESIZE_SHAPE_LIST)
    
    image_batch, label_batch = tf.train.shuffle_batch([image, label], batch_size=BATCH_SIZE, capacity=10, 
                                                          num_threads=2, min_after_dequeue=2)
    
    return image_batch, label_batch

# Convolutional Neural Network Model

In [ ]:
def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    
    # Setup the filter input shape for tf.nn.conv_2d
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels, num_filters]

    # Initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03), name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    # Setup the convolutional layer operation
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')
    
    # Add the bias
    out_layer += bias

    # Apply a ReLU non-linear activation
    out_layer = tf.nn.relu(out_layer)

    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=strides, padding='SAME')

    return out_layer


def cnn_model(x):
    layer1 = create_new_conv_layer(x, NUM_CHANNELS, NUM_FILTERS, FILTER_SHAPE, POOL_SHAPE, name='layer1')
    layer2 = create_new_conv_layer(layer1, NUM_FILTERS, NUM_FILTERS*2, 
                                   FILTER_SHAPE, POOL_SHAPE, name='layer2') 

    flattened = tf.reshape(layer2, [-1, 7 * 7 * 64])

    # Setup weights and bias values for this layer, then activate with ReLU
    wd1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1000], stddev=0.03), name='wd1')
    bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')
    dense_layer1 = tf.matmul(flattened, wd1) + bd1
    dense_layer1 = tf.nn.relu(dense_layer1)

    # Another layer with softmax activations
    wd2 = tf.Variable(tf.truncated_normal([1000, NUM_CLASSES], stddev=0.03), name='wd2')
    bd2 = tf.Variable(tf.truncated_normal([NUM_CLASSES], stddev=0.01), name='bd2')
    dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
    y_ = tf.nn.softmax(dense_layer2)

    return y_

# Training Convolutional Neural Network

In [ ]:
def train_cnn(train_size):
    
    image, label = read_from_tfrecords('train')

    with tf.Session() as sess:
        sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        
        for epoch in range(NUM_EPOCHS):
            epoch_loss = 0
            for _ in range(int(train_size/BATCH_SIZE)-1): 
                img, lbl = sess.run([image, label])
                prediction = cnn_model(img)
                cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, 
                                                                                 labels=tf.one_hot(lbl, NUM_CLASSES)))
                optimizer = tf.train.AdamOptimizer().minimize(cost)
                epoch_loss += cost
                
            print('Epoch', epoch, 'completed out of', NUM_EPOCHS, '. Loss:', epoch_loss)
         
        coord.request_stop()
        coord.join(threads)

# Main

In [ ]:
# Collect data from "Dataset/", shuffle, and split it into training, validation & testing set.
data_size, train_img, train_labels, val_img, val_labels, test_img, test_labels = collect_split_data()

# Creates train.tfrecords, val.tfrecords & test.tfrecords
create_tfrecords(train_img, train_labels, val_img, val_labels, test_img, test_labels)

# Training of CNN
train_cnn(data_size*TRAIN_VAL_TEST['train']) 